In [11]:
import random
import math

# Function to parse Solomon instance from file
def parse_solomon_instance(file_path):
    customers = []
    with open(file_path, 'r') as f:
        lines = f.readlines()
        vehicle_info = lines[4].split()
        num_vehicles = int(vehicle_info[0])  # Max number of vehicles
        capacity = int(vehicle_info[1])      # Vehicle capacity
        
        for line in lines[9:]:
            data = line.split()
            if len(data) == 7:
                customer = {
                    'id': int(data[0]),
                    'x': float(data[1]),
                    'y': float(data[2]),
                    'demand': float(data[3]),
                    'ready_time': float(data[4]),
                    'due_date': float(data[5]),
                    'service_time': float(data[6])
                }
                customers.append(customer)
    
    depot = customers[0]  # Customer 0 is the depot
    return customers, depot, capacity, num_vehicles

# Distance calculation
def calculate_distance(c1, c2):
    return math.sqrt((c1['x'] - c2['x'])**2 + (c1['y'] - c2['y'])**2)

# Decode particle position into routes
def decode_position(position, customers, depot, capacity, max_vehicles):
    customer_indices = list(range(1, len(customers)))  # Exclude depot (0)
    sorted_indices = sorted(customer_indices, key=lambda i: position[i-1], reverse=True)
    
    routes = []
    current_route = []
    current_capacity = 0
    current_time = 0
    
    for idx in sorted_indices:
        customer = customers[idx]
        if len(routes) >= max_vehicles and current_route:  # If max vehicles reached, stop adding new routes
            break
        if current_capacity + customer['demand'] <= capacity:
            last_node = depot if not current_route else customers[current_route[-1]]
            travel_time = calculate_distance(last_node, customer)
            arrival_time = current_time + travel_time
            
            if arrival_time <= customer['due_date']:
                wait_time = max(0, customer['ready_time'] - arrival_time)
                current_time = max(arrival_time, customer['ready_time']) + customer['service_time']
                current_route.append(idx)
                current_capacity += customer['demand']
            else:
                if current_route:
                    routes.append(current_route)
                current_route = [idx]
                current_capacity = customer['demand']
                current_time = customer['ready_time'] + customer['service_time']
        else:
            if current_route:
                routes.append(current_route)
            current_route = [idx]
            current_capacity = customer['demand']
            current_time = customer['ready_time'] + customer['service_time']
    
    if current_route and len(routes) < max_vehicles:
        routes.append(current_route)
    
    # Calculate total distance
    total_distance = 0
    for route in routes:
        prev = depot
        for idx in route:
            total_distance += calculate_distance(prev, customers[idx])
            prev = customers[idx]
        total_distance += calculate_distance(prev, depot)  # Return to depot
    
    return routes, total_distance

# Fitness function with vehicle restriction
def fitness(routes, total_distance, num_customers, max_vehicles):
    num_vehicles = len(routes)
    served_customers = sum(len(route) for route in routes)
    penalty_unserved = 10000 * (num_customers - served_customers)  # Penalize unserved customers
    penalty_vehicles = 100000 * max(0, num_vehicles - max_vehicles)  # Penalize exceeding max vehicles
    return num_vehicles * 1000 + total_distance + penalty_unserved + penalty_vehicles

# Particle class
class Particle:
    def __init__(self, num_customers):
        self.position_i = [random.uniform(0, 1) for _ in range(num_customers-1)]
        self.velocity_i = [random.uniform(-0.1, 0.1) for _ in range(num_customers-1)]
        self.pos_best_i = self.position_i.copy()
        self.err_best_i = float('inf')
        self.err_i = float('inf')

# PSO algorithm
def pso_vrptw(customers, depot, capacity, num_particles, max_iter, max_vehicles):
    swarm = [Particle(len(customers)) for _ in range(num_particles)]
    global_best_pos = None
    global_best_err = float('inf')
    
    for iteration in range(max_iter):
        for particle in swarm:
            routes, distance = decode_position(particle.position_i, customers, depot, capacity, max_vehicles)
            particle.err_i = fitness(routes, distance, len(customers)-1, max_vehicles)
            
            if particle.err_i < particle.err_best_i:
                particle.pos_best_i = particle.position_i.copy()
                particle.err_best_i = particle.err_i
            
            if particle.err_i < global_best_err:
                global_best_pos = particle.position_i.copy()
                global_best_err = particle.err_i
            
            w, c1, c2 = 0.7, 2.0, 2.0
            for d in range(len(particle.position_i)):
                r1, r2 = random.random(), random.random()
                particle.velocity_i[d] = (w * particle.velocity_i[d] +
                                         c1 * r1 * (particle.pos_best_i[d] - particle.position_i[d]) +
                                         c2 * r2 * (global_best_pos[d] - particle.position_i[d]))
                particle.position_i[d] = max(0, min(1, particle.position_i[d] + particle.velocity_i[d]))
    
    routes, distance = decode_position(global_best_pos, customers, depot, capacity, max_vehicles)
    return routes, distance, len(routes)

# Main execution
text_file = 'Data\\c101.txt'
customers, depot, capacity, max_vehicles = parse_solomon_instance(text_file)

# Run PSO with 100 particles and 1000 iterations
routes, distance, num_vehicles = pso_vrptw(customers, depot, capacity, 100, 1000, max_vehicles)

# Output results
print(f"Instance: c101")
print(f"Maximum Number of Vehicles: {max_vehicles}")
print(f"Number of Vehicles Used: {num_vehicles}")
print(f"Total Distance: {distance:.2f}")
print(f"Routes:")
for i, route in enumerate(routes, 1):
    print(f"  Vehicle {i}: {route}")

Instance: c101
Maximum Number of Vehicles: 25
Number of Vehicles Used: 25
Total Distance: 2432.09
Routes:
  Vehicle 1: [5, 8]
  Vehicle 2: [17, 25, 27, 28]
  Vehicle 3: [30]
  Vehicle 4: [32, 42, 44, 46, 48, 49]
  Vehicle 5: [67, 75]
  Vehicle 6: [81, 83, 84, 85, 88]
  Vehicle 7: [92, 93, 97]
  Vehicle 8: [98]
  Vehicle 9: [43, 21]
  Vehicle 10: [63, 72, 9]
  Vehicle 11: [13, 74, 6, 2]
  Vehicle 12: [94]
  Vehicle 13: [96, 53, 79, 50]
  Vehicle 14: [29]
  Vehicle 15: [95, 19, 15]
  Vehicle 16: [3, 10, 45, 69]
  Vehicle 17: [20, 86, 58, 68, 22]
  Vehicle 18: [24]
  Vehicle 19: [87, 34, 52]
  Vehicle 20: [57, 54, 60, 51]
  Vehicle 21: [33, 36]
  Vehicle 22: [41, 100, 89]
  Vehicle 23: [78, 82, 91]
  Vehicle 24: [40, 11]
  Vehicle 25: [71, 39]
